# Customer Lifetime Value (CLV) Prediction

In this chapter, you will learn the basics of Customer Lifetime Value (CLV) and its different calculation methodologies. You will harness this knowledge to build customer level purchase features to predict next month's transactions using linear regression.

## CLV basics

CLV is how much a company expects to earn from an average customer in a lifetime.
It can be hitorical: where we sum up each customer's profit an get the actual customer value.
or it can be predicted.

**Historical CLV**

\begin{equation*}
\text{Historical CLV} = (\text{Revenue 1} + \text{Revenue 2} + \cdots + \text{Revenue N}) * \text{Profit Margin}
\end{equation*}

**Basic CLV formula**

\begin{equation*}
\text{CLV} = \text{Avg. Revenue} * \text{Profit Margin} * \text{Avg. Lifespan}
\end{equation*}

**Granular CLV formula**

\begin{equation*}
\text{CLV} = (\text{Avg. revenue per purchase} * \text{Avg. Frequency} * \text{Profit Margin}) * \text{Avg. Lifespan}
\end{equation*}

**Traditional CLV formula**

\begin{equation*}
\text{CLV} = (\text{Avg. Revenue} * \text{Profit Margin}) * \frac{\text{Retention Rate}}{\text{Churn Rate}}
\end{equation*}

**Excercise 1**
```python
# Extract cohort sizes from the first column of cohort_counts
cohort_sizes = cohort_counts.iloc[:,0]

# Calculate retention by dividing the counts with the cohort sizes
retention = cohort_counts.divide(cohort_sizes, axis=0)

# Calculate churn
churn = 1 - retention

# Print the retention table
print(retention)
```

**Excercise 2**
```python
# Calculate the mean retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate the mean churn rate
churn_rate = churn.iloc[:,1:].mean().mean()

# Print rounded retention and churn rates
print('Retention rate: {:.2f}; Churn rate: {:.2f}'.format(retention_rate, churn_rate))
```

## Calculting and projecting CLV

**Basic CLV calculation**
```python
#Calculate monthly spend per customer
monthly revenue = online.grouby(['CustomerID', 'InvoiceMonth'])['TotalSum'].sum().mean()

#Calculate average monthly spend
monthly_revenue = np.mean(monthly_revenue)

#Define Lifespan to 36 months
lifespan_months = 36

#Calculate basic CLV
clv_basic = monthly_revenue * lifespan_months

#Print basic CLV value
print('Average basic CLV is {:.1f} USD.'.format(clv_basic))
```
**Granular CLV calculation**
```python
#Calculate average revenue per invoice
revenue_per_purchase = online.groupby(['InvoiceNo'])['TotalSum'].mean().mean()

#Calculate average number of uniques invoices per customer per month
freq = online.groupby(['CustomerId', 'InvoiceMonth'])['InvoiceNo'].nunique().mean()

#Define Lifespan to 36 months
lifespan_months = 36

#Calculate granular CLV
clv_granular = revenue_per_purchase * freq * lifespan_months

#Print basic CLV value
print('Average granular CLV is {:.1f} USD.'.format(clv_granular))
```

**Traitional CLV calculation**
```python
# Calculate monthly spend per customer
monthly_revenue = online.groupby(['CusotmerID', 'InvoiceMonth'])['TotalSum'].sum().mean()

#Calculate average monthly retention rate
retention_rate = retention.iloc[:, 1:].mean().mean()

#Calculate average monthly churn rate
churn_rate = 1 - retention_rate

clv_traditional = monthly_revenue * ( retention_rate / churn_rate)

#Print traditional CLV and the retention rate values
print('Average traditional CLV is {:.1f} USD at {:.1f} % retention_rate.'.format(clv_tradicional, retention_rate * 100))
```

## DataPreparation for purchase prediction

**Build Features based on RFM**
```python
# Define the snapshot date
NOW = dt.datetime(2011,11,1)

# Calculate recency by subtracting current date from the latest InvoiceDate
features = online_X.groupby('CustomerID').agg({
  'InvoiceDate': lambda x: (NOW - x.max()).days,
  # Calculate frequency by counting unique number of invoices
  'InvoiceNo': pd.Series.nunique,
  # Calculate monetary value by summing all spend values
  'TotalSum': np.sum,
  # Calculate average and total quantity
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']
```

**Define Target Variable**
```python
# Build a pivot table counting invoices for each customer monthly
cust_month_tx = pd.pivot_table(data=online, values='InvoiceNo',
                               index=['CustomerID'], columns=['InvoiceMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)

# Store November 2011 data column name as a list
target = ['2011-11']

# Store target value as `Y`
Y = cust_month_tx[target]
```

**Split data to training and testing**

```python
# Store customer identifier column name as a list
custid = ['CustomerID']

# Select feature column names excluding customer identifier
cols = [col for col in features.columns if col not in custid]

# Extract the features as `X`
X = features[cols]

# Split data to training and testing
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25, random_state=99)
```

## Predicting Customer Transactions

**Regression Performance Metrics**
- Root Mean Squared Error (RMSE) - Square root of the average squared difference between prediction and actuals
- Mean Absolute Error (MAE) - Average absolute difference between prediction and actuals
- Mean Absolue Percentage Error (MAPE) - Average percentage difference between prediction and actuals (actuals can't be zeros)

**Predict next month transactions**

```python
# Initialize linear regression instance
linreg = LinearRegression()

# Fit the model to training dataset
linreg.fit(train_X, train_Y)

# Predict the target variable for training data
train_pred_Y = linreg.predict(train_X)

# Predict the target variable for testing data
test_pred_Y = linreg.predict(test_X)
```

**Measure Model Fit**
```python
# Calculate root mean squared error on training data
rmse_train = np.sqrt(mean_squared_error(train_Y, train_pred_Y))

# Calculate mean absolute error on training data
mae_train = mean_absolute_error(train_Y, train_pred_Y)

# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))

# Calculate mean absolute error on testing data
mae_test = mean_absolute_error(test_Y, test_pred_Y)

# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))
```

**Explore Model Coefficients**
```python
# Import `statsmodels.api` module
import statsmodels.api as sm

# Initialize model instance on the training data
olsreg = sm.OLS(train_Y, train_X)

# Fit the model
olsreg = olsreg.fit()

# Print model summary
print(olsreg.summary())



                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.488
Model:                            OLS   Adj. R-squared (uncentered):              0.487
Method:                 Least Squares   F-statistic:                              480.3
Date:                Tue, 05 May 2020   Prob (F-statistic):                        0.00
Time:                        05:19:51   Log-Likelihood:                         -2769.8
No. Observations:                2529   AIC:                                      5550.
Df Residuals:                    2524   BIC:                                      5579.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
recency            0.0002      0.000      1.701      0.089   -2.92e-05       0.000
frequency          0.1316      0.003     38.000      0.000       0.125       0.138
monetary        1.001e-06   3.59e-05      0.028      0.978   -6.95e-05    7.15e-05
quantity_avg       0.0001      0.000      0.803      0.422      -0.000       0.000
quantity_total    -0.0001   5.74e-05     -2.562      0.010      -0.000   -3.45e-05
==============================================================================
Omnibus:                      987.494   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5536.657
Skew:                           1.762   Prob(JB):                         0.00
Kurtosis:                       9.334   Cond. No.                         249.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

```